In [1]:
import numpy as np
import polars as pl

In [3]:
threshold = 271
lt = 183

lower_pwr_bound = [0.0, 0.55, 0.75, 0.90, 1.00, 1.10, 1.35]
lower_lt_bound = [0.0, 0.7, 0.87, 0.95, 1.0, 1.05, None]

def shift_left(array: list) -> list:
    size = len(array)
    return [a for a in array[1:]] + [None]


def concrete_bound(pct_col_name: str, metric: int, upper: bool = False) -> pl.Expr:
    base_col = (
        (pl.col(pct_col_name) * pl.lit(metric))
        .cast(pl.Int16)
        .alias(pct_col_name.rstrip("_pct"))
    )
    return base_col - pl.lit(1) if upper else base_col


systm_zones_df = pl.DataFrame(
    {
        "zone_name": [
            "Active Recovery",
            "Endurance",
            "Tempo",
            "Sub Lactate",
            "Supra Lactate",
            "VO2 Max",
            "Anerobic Capacity/NM",
        ],
        "th_lower_bound_pct": lower_pwr_bound,
        "th_upper_bound_pct": shift_left(lower_pwr_bound),
        "lt_lower_bound_pct": lower_lt_bound,
        "lt_upper_bound_pct": shift_left(lower_lt_bound),
    }
).select(
    "*",
    concrete_bound("th_lower_bound_pct", threshold),
    concrete_bound("th_upper_bound_pct", threshold, upper=True),
    concrete_bound("lt_lower_bound_pct", lt),
    concrete_bound("lt_upper_bound_pct", lt, upper=True),
)

systm_zones_df


zone_name,th_lower_bound_pct,th_upper_bound_pct,lt_lower_bound_pct,lt_upper_bound_pct,th_lower_bound,th_upper_bound,lt_lower_bound,lt_upper_bound
str,f64,f64,f64,f64,i16,i16,i16,i16
"""Active Recover…",0.0,0.55,0.0,0.7,0,148,0,127
"""Endurance""",0.55,0.75,0.7,0.87,149,202,128,158
"""Tempo""",0.75,0.9,0.87,0.95,203,242,159,172
"""Sub Lactate""",0.9,1.0,0.95,1.0,243,270,173,182
"""Supra Lactate""",1.0,1.1,1.0,1.05,271,297,183,191
"""VO2 Max""",1.1,1.35,1.05,null,298,364,192,null
"""Anerobic Capac…",1.35,null,null,null,365,null,null,null
